In [4]:
import tensorflow as tf
import mlflow
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import os
import numpy as np
import mlflow
from PIL import Image
import cv2

In [8]:
import splitfolders

input_folder = 'data/'
splitfolders.ratio(input_folder, output="output_folder", seed=42, ratio=(.7, .2, .1), group_prefix=None)  

Copying files: 400 files [00:00, 761.85 files/s]


In [87]:
# Path to your folder containing images
train_path="v_data/train"
test_path="v_data/test"
val_path="v_data/val"

In [ ]:
# Initialize an empty list to hold the image arrays
x_train = []

# Iterate over all image files in the folder
for filename in os.listdir(train_path):
    sub_path=train_path+"/"+ filename
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_train.append(img_arr)

# Convert the list of arrays into a NumPy array
x_train_np = np.array(x_train)

# Print the shape of the resulting array (should be something like (num_images, 256, 256, 3))
print("Shape of the NumPy array:", x_train_np.shape)

In [ ]:
x_test=[]
for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_test.append(img_arr)

x_val=[]
for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_val.append(img_arr)

# Convert the list of arrays into a NumPy array
x_test_np = np.array(x_test)
x_val_np = np.array(x_val)
# Print the shape of the resulting array (should be something like (num_images, 256, 256, 3))
print("Shape of the NumPy array:", x_test_np.shape)
print("Shape of the NumPy array:", x_val_np.shape)

In [90]:
#Normalizing the data
train_x=x_train_np/255.0 
test_x=x_test_np/255.0 
val_x=x_val_np/255.0

In [131]:
# ImageDataGenerator for data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1.0/255)
print(train_datagen)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [132]:
# Load and preprocess the training data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    shuffle=True,
    class_mode='categorical'  
)

# Load and preprocess the validation data
validation_generator = validation_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    shuffle=False,
    class_mode='categorical'
)

print(train_datagen)
print(train_generator)

Found 400 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [133]:
#Build a model
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(224, 224,3)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation = 'softmax')
    ])

c:\Users\Divyaprabha\.conda\envs\ml-pipeline\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [134]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',  # Use 'binary_crossentropy' for binary classification
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_15 (Flatten)            │ (None, 150528)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 128)            │    19,267,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,267,970 (73.50 MB)

 Trainable params: 19,267,970 (73.50 MB)

 Non-trainable params: 0 (0.00 B)

In [135]:
model.fit(train_generator, 
          steps_per_epoch = train_generator.samples // 2,
          epochs=20, 
         validation_data=validation_generator,
         validation_steps=validation_generator.samples // 2,
         verbose=1)

model.save('my_model.keras')

2024/09/13 11:42:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ed40abfdd95146be91065b5f1152ec5e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow
2024/09/13 11:42:20 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
2024/09/13 11:42:20 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.


Epoch 1/20


c:\Users\Divyaprabha\.conda\envs\ml-pipeline\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 13/200 ━━━━━━━━━━━━━━━━━━━━ 39s 213ms/step - accuracy: 0.5333 - loss: 71.7351

c:\Users\Divyaprabha\.conda\envs\ml-pipeline\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


...

200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.8398 - loss: 0.7832 - val_accuracy: 0.8200 - val_loss: 1.6993
Epoch 13/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9579 - loss: 0.1393 - val_accuracy: 0.7400 - val_loss: 2.2051
Epoch 14/20
 13/200 ━━━━━━━━━━━━━━━━━━━━ 38s 204ms/step - accuracy: 0.9551 - loss: 0.1612

200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.9667 - loss: 0.1253 - val_accuracy: 0.7800 - val_loss: 2.1315
Epoch 15/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9753 - loss: 0.0889 - val_accuracy: 0.7600 - val_loss: 3.0142
Epoch 16/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9695 - loss: 0.1021 - val_accuracy: 0.7400 - val_loss: 3.2209
Epoch 17/20
 13/200 ━━━━━━━━━━━━━━━━━━━━ 37s 201ms/step - accuracy: 0.9458 - loss: 0.1590

200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9591 - loss: 0.1411 - val_accuracy: 0.8000 - val_loss: 2.2859
Epoch 18/20
 13/200 ━━━━━━━━━━━━━━━━━━━━ 41s 220ms/step - accuracy: 0.9967 - loss: 0.0079

200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9974 - loss: 0.0080 - val_accuracy: 0.8000 - val_loss: 1.8526
Epoch 19/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.8000 - val_loss: 2.0431
Epoch 20/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9258 - loss: 0.5866 - val_accuracy: 0.5800 - val_loss: 9.0677


2024/09/13 11:44:33 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: '>=' not supported between instances of 'slice' and 'int'
2024/09/13 11:44:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


In [136]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
train_loss, train_accuracy = model.evaluate(train_generator)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5012 - loss: 10.7679
Validation Accuracy: 58.00%
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.6154 - loss: 4.4532
Train Accuracy: 63.50%


In [137]:
# Prediction using trained model
print(type(test_x))
print(test_x.shape)

<class 'numpy.ndarray'>
(50, 224, 224, 3)


In [138]:
#Batch Prediction
predictions = model.predict(test_x)

print(predictions)
# If you want to get the predicted class for each image
predicted_classes = np.argmax(predictions, axis=1)

# Print predicted classes
print("Predicted classes:", predicted_classes)



2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step
[[7.6082373e-16 1.0000000e+00]
 [2.9676914e-14 1.0000000e+00]
 [1.4644294e-23 1.0000000e+00]
 [5.4464674e-01 4.5535326e-01]
 [7.7723709e-15 1.0000000e+00]
 [1.8851539e-07 9.9999976e-01]
 [4.6412831e-18 1.0000000e+00]
 [2.8825079e-11 1.0000000e+00]
 [2.6827872e-12 1.0000000e+00]
 [5.2743114e-04 9.9947256e-01]
 [1.7912515e-11 1.0000000e+00]
 [3.1518654e-04 9.9968481e-01]
 [1.7154188e-16 1.0000000e+00]
 [4.4323945e-10 1.0000000e+00]
 [3.7178867e-09 1.0000000e+00]
 [6.3391238e-01 3.6608765e-01]
 [2.9567578e-11 1.0000000e+00]
 [5.0978433e-17 1.0000000e+00]
 [5.5986398e-01 4.4013599e-01]
 [5.1800029e-08 1.0000000e+00]
 [6.9963824e-14 1.0000000e+00]
 [1.5862004e-16 1.0000000e+00]
 [6.9757589e-13 1.0000000e+00]
 [1.5741737e-25 1.0000000e+00]
 [1.9966062e-06 9.9999797e-01]
 [1.3172152e-16 1.0000000e+00]
 [8.2751448e-19 1.0000000e+00]
 [3.2821200e-12 1.0000000e+00]
 [2.6534493e-12 1.0000000e+00]
 [6.8322395e-25 1.0000000e+00]
 [1.2663476e-24 1.0000000e+00]
 

In [139]:
#Single image Prediction
pred_path = '20.jpg'
img = image.load_img(pred_path, target_size=(224,224))

img_array = image.img_to_array(img)
img_array = img_array / 255.0 

img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
print(predictions) 

predicted_class = np.argmax(predictions, axis=1)
print(f"Predicted class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
[[6.035186e-13 1.000000e+00]]
Predicted class: [1]


In [ ]:
from mlflow.models.signature import infer_signature
model_signature = infer_signature(test_x, model.predict(test_x))
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
from datetime import datetime
input_data_path = 'v_data'
exp_timestamp = datetime.now().strftime("%Y%m%d")
exp_name = "custom_ds" + exp_timestamp
print(exp_name)

run_timestamp = datetime.now().strftime("%Y%m%d--%H%M%S")
run_name = "custom_ds_run"+ run_timestamp
print(run_name)

#mlflow.tensorflow.autolog(log_every_epoch=2)

mlflow.set_experiment(exp_name)
with mlflow.start_run(run_name=run_name):
        #mlflow.log_artifact(input_data_path, artifact_path="input_data")
        mlflow.log_param("batch_size", 2)
        #mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("epochs", 20)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("val_loss", val_loss)
        mlflow.log_metric("val_accuracy", val_accuracy)
        
        mlflow.tensorflow.log_model(model, "custom_ds", signature=model_signature,registered_model_name='custom_ds_model')
mlflow.end_run()
